# CVA Calculation for an Interest Rate Swap

In this report we do following:
1. Describe how to calculate the credit valuation adjustment taken by Counterpart A to the price of an interest rate swap using credit spreads for Counterparty B. 
2. Plot MtM values (a good plot will show results from many simulations) and produce :

    * A smoothed Expected Exposure profile. While EE is defined as Max($MtM_{\tau}$, 0), we experiment with exposure distributions(s) at each tenor. Produce smoothed Potential Future Exposure(s) using such indicators as

    * The median of positive exposure
    
    * 97.5th percentile

#### Input for IRS Valuations are :

1. Forward LIBORs
2. Discounting Factors (DF)
3. Default probabilities (PDs)

Assumptions
* The IRS is assumed to be written on a 6M LIBOR $L_{6M}$ expiring in 5Y, hence the payment frequency is $\tau=0.5$
* The notional is assumed to be $N=1$
* Recovery rate 40%

 ### Default Probabilities (PDs)

Default Probabilities are calculated from CDS spreads using the bootstrapping in excel 'PD CDS Bootstrapping.xlsx.'

In [2]:
from matplotlib.ticker import FuncFormatter

import pandas as pd
# Import statsmodels equivalents to validate results
from statsmodels.tsa.api import VAR
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.tsatools import (lagmat, add_trend)
from statsmodels.tsa.stattools import adfuller
import numpy as np
import math
from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pylab as pylab


data2 = pd.read_csv("inputcva.csv", index_col=0)
output = data2.to_string(formatters={'Lambda': '{:,.4%}'.format, 'PD': '{:,.4%}'.format,'P': '{:,.4%}'.format})
print(output)

           CDS        DF  Lambda      PD         P
Tenor                                             
0.0        NaN  1.000000    nan% 0.0000% 100.0000%
0.5    114.400  0.995835 1.8976% 0.9443%  99.0557%
1.0    133.770  0.990963 2.2197% 1.2510%  97.8047%
1.5    167.180  0.985697 2.7798% 1.8887%  95.9161%
2.0    200.590  0.980105 3.3452% 2.3875%  93.5285%
2.5    233.965  0.974101 3.9174% 2.8579%  90.6707%
3.0    267.340  0.967832 4.4994% 3.2974%  87.3732%
3.5    296.545  0.961232 5.0170% 3.4776%  83.8957%
4.0    325.750  0.954239 5.5471% 3.7947%  80.1009%
4.5    353.200  0.946899 6.0576% 3.9605%  76.1405%
5.0    380.650  0.939187 6.5842% 4.1914%  71.9490%


## Forward LIBORs


* LIBOR  indicates money-market rates for actual loans between banks for upto 18 M

* OIS targets the Federal Funds Rate, an average over the period. The spread for LIBOR fixings Vs OIS prices reveals the short-term credit risk in the financial system.

#### OIS Discount factors

* Observe the difference between DF taken under the forward measure and $DF_{OIS}$ representing the risk-neutral measure Q.

Used Bank of England data for Forward Curve from BOE website http://www.bankofengland.co.uk and used 5 year data from Tab '1. fwds, short end' rest 20 years from Tab '2. fwd curve' from sheet ukblc05_mdaily.xlsx and ukblc16_mdaily.xlsx. Data sampled from 13th Sep 2015 till 20th July 2017 and kept in sheet HJM Model MC.xlsm Tab 'FwdCurve'

Following step was taken to find eigen values using Principal Components Analysis:
1. Calcualted difference of Forward Curve data in Tab 'FwdCurve' and kept in Tab 'Difference'
2. Calcualted Covariance of Difference in Tab 'Difference' and kept it in Tab 'Covariance'
3. Calculated Eigenvalues based on 'Covariance' Tab and kept in PCFV tab.
4. Calcualted EigenVectors using Jacobi algorithm.
5. Sorted Eigenvalues and found the top 3 eigen values.

|             | Tenor | Eigenvalue | Cum. R^2 |
|-------------|-------|------------|----------|
| 1st largest PC| 7.0	| 0.002370368 |	8.98E-01 |
| 2nd largest PC| 2.0	| 0.000121736 | 9.45E-01 |
| 3rd largest PC| 25.0 | 7.71771E-05 | 9.738E-01 |

* Selected Principal Components as top 3 eigen values and calucated Principal Components volatility functions as below in Tab 'Volatility':
$$
Vol_i = \sqrt{\lambda_i} \space e(i) \space\space\space \forall \space i=1,2,3
$$
